# QROM

Quantum read-only memory.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `QROM`
Bloq to load `data[l]` in the target register when the selection stores an index `l`.

## Overview
The action of a QROM can be described as
$$
        \text{QROM}_{s_1, s_2, \dots, s_K}^{d_1, d_2, \dots, d_L}
        |s_1\rangle |s_2\rangle \dots |s_K\rangle
        |0\rangle^{\otimes b_1} |0\rangle^{\otimes b_2} \dots |0\rangle^{\otimes b_L}
        \rightarrow
        |s_1\rangle |s_2\rangle \dots |s_K\rangle
        |d_1[s_1, s_2, \dots, s_k]\rangle
        |d_2[s_1, s_2, \dots, s_k]\rangle \dots
        |d_L[s_1, s_2, \dots, s_k]\rangle
$$

There two high level parameters that control the behavior of a QROM are -

1. Shape of the classical dataset to be loaded ($\text{data.shape} = (S_1, S_2, ..., S_K)$).
2. Number of distinct datasets to be loaded ($\text{data.bitsizes} = (b_1, b_2, ..., b_L)$).

Each of these have an effect on the cost of the QROM. The `data_or_shape` parameter stores
either
1. A numpy array of shape $(L, S_1, S_2, ..., S_K)$ when $L$ classical datasets, each of
   shape $(S_1, S_2, ..., S_K)$ and bitsizes $(b_1, b_2, ..., b_L)$ are to be loaded and
   the classical data is available to instantiate the QROM bloq. In this case, the helper
   builder `QROM.build_from_data(data_1, data_2, ..., data_L)` can be used to build the QROM.

2. A `Shaped` object that stores a (potentially symbolic) tuple $(L, S_1, S_2, ..., S_K)$
   that represents the number of classical datasets `L=data_or_shape.shape[0]` and
   their shape `data_shape=data_or_shape.shape[1:]` to be loaded by this QROM. This is used
   to instantiate QROM bloqs for symbolic cost analysis where the exact data to be loaded
   is not known. In this case, the helper builder `QROM.build_from_bitsize` can be used
   to build the QROM.

### Shape of the classical dataset to be loaded.
QROM bloq supports loading multidimensional classical datasets. In order to load a data
set of shape $\mathrm{data.shape} == (P, Q, R, S)$ the QROM bloq needs four selection
registers with bitsizes $(p, q, r, s)$ where
$p,q,r,s=\log_2{P}, \log_2{Q}, \log_2{R}, \log_2{S}$.

In general, to load K dimensional data, we use K named selection registers `(selection0,
selection1, ..., selection{k})` to index and load the data.

The T/Toffoli cost of the QROM scales linearly with the number of elements in the dataset
(i.e. $\mathcal{O}(\mathrm{np.prod(data.shape)}$).

### Number of distinct datasets to be loaded, and their corresponding target bitsize.
To load a classical dataset into a target register of bitsize $b$, the clifford cost of a QROM
scales as $\mathcal{O}(b \mathrm{np.prod}(\mathrm{data.shape}))$. This is because we need
$\mathcal{O}(b)$ CNOT gates to load the ith data element in the target register when the
selection register stores index $i$.

If you have multiple classical datasets `(data_1, data_2, data_3, ..., data_L)` to be loaded
and each of them has the same shape `(data_1.shape == data_2.shape == ... == data_L.shape)`
and different target bitsizes `(b_1, b_2, ..., b_L)`, then one construct a single classical
dataset `data = merge(data_1, data_2, ..., data_L)` where

- `data.shape == data_1.shape == data_2.shape == ... == data_L` and
- `data[idx] = f'{data_1[idx]!0{b_1}b}' + f'{data_2[idx]!0{b_2}b}' + ... + f'{data_L[idx]!0{b_L}b}'`

Thus, the target bitsize of the merged dataset is $b = b_1 + b_2 + \dots + b_L$ and clifford
cost of loading merged dataset scales as
$\mathcal{O}((b_1 + b_2 + \dots + b_L) \mathrm{np.prod}(\mathrm{data.shape}))$.

## Variable spaced QROM
When the input classical data contains consecutive entries of identical data elements to
load, the QROM also implements the "variable-spaced" QROM optimization described in Ref [2].

#### Parameters
 - `data_or_shape`: List of numpy ndarrays specifying the data to load. If the length of this list ($L$) is greater than one then we use the same selection indices to load each dataset. Each data set is required to have the same shape $(S_1, S_2, ..., S_K)$ and to be of integer type. For symbolic QROMs, pass a `Shaped` object instead with shape $(L, S_1, S_2, ..., S_K)$.
 - `selection_bitsizes`: The number of bits used to represent each selection register corresponding to the size of each dimension of the array $(S_1, S_2, ..., S_K)$. Should be the same length as the shape of each of the datasets.
 - `target_bitsizes`: The number of bits used to represent the data signature. This can be deduced from the maximum element of each of the datasets. Should be a tuple $(b_1, b_2, ..., b_L)$ of length `L = len(data)`, i.e. the number of datasets to be loaded.
 - `num_controls`: The number of controls. 

#### References
 - [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662).     Babbush et. al. (2018). Figure 1.
 - [Compilation of Fault-Tolerant Quantum Heuristics for Combinatorial Optimization](https://arxiv.org/abs/2007.07391).     Babbush et. al. (2020). Figure 3.


In [ ]:
from qualtran.bloqs.data_loading.qrom import QROM

### Example Instances

In [ ]:
data = np.arange(5)
qrom_small = QROM([data], selection_bitsizes=(3,), target_bitsizes=(3,))

In [ ]:
data1 = np.arange(5)
data2 = np.arange(5) + 1
qrom_multi_data = QROM([data1, data2], selection_bitsizes=(3,), target_bitsizes=(3, 4))

In [ ]:
data1 = np.arange(9).reshape((3, 3))
data2 = (np.arange(9) + 1).reshape((3, 3))
qrom_multi_dim = QROM([data1, data2], selection_bitsizes=(2, 2), target_bitsizes=(8, 8))

In [ ]:
N, M, b1, b2, c = sympy.symbols('N M b1 b2 c')
qrom_symb = QROM.build_from_bitsize((N, M), (b1, b2), num_controls=c)
qrom_symb

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([qrom_small, qrom_multi_data, qrom_multi_dim, qrom_symb],
           ['`qrom_small`', '`qrom_multi_data`', '`qrom_multi_dim`', '`qrom_symb`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
qrom_small_g, qrom_small_sigma = qrom_small.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(qrom_small_g)
show_counts_sigma(qrom_small_sigma)

In [ ]:
qrom_symb_g, qrom_symb_sigma = qrom_symb.call_graph(generalizer=ignore_split_join)
show_call_graph(qrom_symb_g)
show_counts_sigma(qrom_symb_sigma)

In [ ]:
N, M, b1, b2, c = sympy.symbols('N M b1 b2 c')
qrom_symb = QROM.build_from_bitsize((N, M), (b1, b2), num_controls=c)